In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from shapely.geometry import box

## Investigating existing datasets

<div class="alert alert-block alert-info">
<b>Tip:</b> To investigate datasets in an interactive manner, a streamlit dashboard will be set up. Until this is ready, please find the descriptions below. 
</div>

Gsemantique offers preconfigured access to a total of 11 different datasets that are provided with global (or near global) coverage. The datasets originate from 3 different endpoints (and the Sentinel-1 and Sentinel-2 data sets can be retrieved from two of these providers). The different datasets include the following types of data:  
* original reflectance values (S1, S2, Landsat)
* terrain/topography products (DEM, DSM) 
* hydrological products (HAND, GSW) 
* burned area products (BAM, BAD) 
* landcover products (ESRI, ESA)

Internally, all `Dataset` instances are organised in a `DatasetCatalog` class. An empty `DatasetCatalog` can be imported and instantiated as shown below. The `.load()` method needs to be called to populate the catalog with the data products listed above.

In [3]:
from gsemantique.data.datasets import *
ds_catalog = DatasetCatalog()
ds_catalog.load()

To inspect the data catalog, you can use the following commands.

In [4]:
print(ds_catalog)

DatasetCatalog containing
- 3 providers (catalogs)
- 13 datasets (collections)
- 71 layers (assets)

Datasets:
     provider                   collection        category temporality
0      Planet               sentinel-1-rtc             SAR           s
1      Planet               sentinel-2-l2a   multispectral           s
2      Planet                landsat-c2-l2   multispectral           s
3      Planet               esa-worldcover       landcover           Y
4      Planet           io-lulc-annual-v02       landcover           Y
5      Planet                      nasadem             DEM        None
6      Planet               cop-dem-glo-30             DSM        None
7      Planet               modis-64A1-061  fire detection           M
8      Planet               modis-14A2-061  fire detection           D
9      Planet                      jrc-gsw  hydrogeography        None
10  Element84               sentinel-2-l2a   multispectral           s
11        ASF  sentinel-1-global-cohe

In [5]:
ds_catalog.parse_as_table(keys=None)

,category,collection,copyright,endpoint,info,layout_bands,layout_file,layout_keys,provider,spatial_extent,src,temporal_extent,temporality,n_bands
0,SAR,sentinel-1-rtc,CC BY 4.0,https://planetarycomputer.microsoft.com/api/st...,Sentinel-1 represent radar imaging (SAR) satel...,"{'s1_amp_vv': {'name': 'vv', 'description': 'G...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, reflectance, s1_amp_vv), (Planet, re...",Planet,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2014-10-10 00:28:21+00:00, None]",s,4
1,multispectral,sentinel-2-l2a,Copernicus Sentinel Data Terms,https://planetarycomputer.microsoft.com/api/st...,The Sentinel-2 program provides global imagery...,"{'s2_band01': {'name': 'B01', 'description': '...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, reflectance, s2_band01), (Planet, re...",Planet,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2015-06-27 10:25:31+00:00, None]",s,13
2,multispectral,landsat-c2-l2,Public Domain (https://www.usgs.gov/emergency-...,https://planetarycomputer.microsoft.com/api/st...,"Landsat Collection 2 Level-2 Science Products,...","{'lndst_coastal': {'name': 'coastal', 'descrip...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, reflectance, lndst_coastal), (Planet...",Planet,"[-180.0, -90.0, 180.0, 90.0]",https://planetarycomputer.microsoft.com/datase...,"[1982-08-22 00:00:00+00:00, None]",s,10
3,landcover,esa-worldcover,Creative Commons Attribution 4.0 International...,https://planetarycomputer.microsoft.com/api/st...,The European Space Agency (ESA) WorldCover pro...,"{'esa_lc': {'name': 'map', 'description': 'ESA...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, classification, esa_lc)]",Planet,"[-180.0, -60.0, 180.0, 83.0]",https://planetarycomputer.microsoft.com/datase...,"[2020-01-01 00:00:00+00:00, 2021-12-31 23:59:5...",Y,1
4,landcover,io-lulc-annual-v02,Creative Commons BY-4.0,https://planetarycomputer.microsoft.com/api/st...,Time series of annual global maps of land use ...,"{'impact_lc': {'name': 'data', 'description': ...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, classification, impact_lc)]",Planet,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2017-01-01 00:00:00+00:00, 2024-01-01 00:00:0...",Y,1
5,DEM,nasadem,Public Domain (https://lpdaac.usgs.gov/data/da...,https://planetarycomputer.microsoft.com/api/st...,NASADEM provides global topographic data at 1 ...,"{'dem': {'name': 'elevation', 'description': '...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, topography, dem)]",Planet,"[-179.0, -56.0, 179.0, 61.0]",https://planetarycomputer.microsoft.com/datase...,"[2000-02-20 00:00:00+00:00, 2000-02-20 00:00:0...",None,1
6,DSM,cop-dem-glo-30,None,https://planetarycomputer.microsoft.com/api/st...,The Copernicus DEM is a digital surface model ...,"{'dsm': {'name': 'data', 'description': 'Digit...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, topography, dsm)]",Planet,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2021-04-22 00:00:00+00:00, 2021-04-22 00:00:0...",None,1
7,fire detection,modis-64A1-061,None,https://planetarycomputer.microsoft.com/api/st...,The Terra and Aqua combined MCD64A1 Version 6....,"{'m_burn_date': {'name': 'Burn_Date', 'descrip...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, burned_mapping, m_burn_date), (Plane...",Planet,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2000-11-01 00:00:00+00:00, None]",M,3
8,fire detection,modis-14A2-061,None,https://planetarycomputer.microsoft.com/api/st...,The Moderate Resolution Imaging Spectroradiome...,"{'w_burn_qa': {'name': 'QA', 'description': 'P...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, burned_mapping, w_burn_qa), (Planet,...",Planet,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2000-02-18 00:00:00+00:00, Non

With the `Dataset` instance, a core set of high-level overview information. To search for and work with a data set, more detailed information on its different data layers and their charactersitics is needed. Following the conceptual framework of semantique, this information is captured in a so-called dataset layout. To quote from the documentation of [semantique](https://zgis.github.io/semantique/_notebooks/recipes.html#Representing-an-EO-data-cube):

>In semantique, an EO data cube is always represented by its layout. The layout is a JSON-structured file containing a metadata object for each data layer in the cube. The software expert should distribute it alongside the EO data cube, such that the application expert can use it to initialize a Python object that represents the cube.

The layout file for the pre-configured datasets listed above is part of the gsemantique package (see [here](../gsemantique/data/layout.json)). It is organised hierarchically with dataset providers as top-level keys, e.g. (Planet, reflectance, s1_amp_vv). The layout information is bound to `Dataset` instances with the layout-specific attributes "layout_file", "layout_keys", "layout_bands". The "layout_bands" information is directly parsed from the layout file.

In [6]:
layout_bands = ds_catalog.filter(collection="jrc-gsw")["layout_bands"].iloc[0]
layout_bands

{'change': {'name': 'change',
  'description': 'Change in water occurrence between the two periods (1984-1999) and (2000-2020).',
  'type': 'continuous',
  'dtype': 'uint8',
  'values': {},
  'reference': ['Planet', 'hydrogeography', 'change']},
 'extent': {'name': 'extent',
  'description': 'Binary indicator of whether water was ever present (from 1984-2020).',
  'type': 'binary',
  'dtype': 'binary',
  'values': {},
  'reference': ['Planet', 'hydrogeography', 'extent']},
 'occurrence': {'name': 'occurrence',
  'description': 'Frequency with which water was present from March 1984 to December 2020.',
  'type': 'continuous',
  'dtype': 'uint8',
  'values': {},
  'reference': ['Planet', 'hydrogeography', 'occurrence']},
 'transitions': {'name': 'transitions',
  'description': 'Categorical change in surface water status from 1984 to 2020.',
  'type': 'discrete',
  'dtype': 'uint8',
  'values': [{'id': 1, 'label': 'permanent', 'description': 'Permanent'},
   {'id': 2, 'label': 'new perman

## Searching for data

In order to build on-demand STACCubes, one needs to have a list of items at hand that are to be ingested into the cube. To this end, semantique offers the `Finder` class. Its initialisation requires a dataset catalog and the spatio-temporal extent of interest. Subsequently, you can call its .search_man() method to perform a pystac-client based search for items based on a given layer_key. The results are then captured in the property item_coll as shown below. 

In [7]:
?Finder

Object `Finder` not found.


In [8]:
# get the logger to investigate the logs 
import logging
import sys

logger = logging.getLogger("gsemantique.data.search")
logger.setLevel(logging.INFO)
stream_handler = logging.StreamHandler()
logger.addHandler(stream_handler)

In [9]:
from gsemantique.data.search import Finder

t_start, t_end = "2020-01-01", "2020-01-05"
xmin, ymin, xmax, ymax = 142.13, -34.2, 142.18, -34.15
aoi = box(xmin, ymin, xmax, ymax)
layer_key = ("Planet", "reflectance", "lndst_qa")

fdr = Finder(ds_catalog, t_start, t_end, aoi)
fdr.search_man(layer_key)
fdr.item_coll

Found 1 datasets


Using the manual search function, requires to have a layer_key, i.e. a data layout specific attribute. As a user, you usually only create a recipe on the semantic level. Additionally, you do have access to some mapping of entities and the layout file (both of which were possibly provided to you in a preconfigured manner by an EO expert). You most certainly don't care about the layer key but simply want to have the data needed for you recipe being automatically searched for. To this end, the Finder has a second method called .search_auto. This method takes a recipe, mapping and datacube as an input and automatically resolves the data layer references needed to evaluate the recipe. The search results are again stored in the .item_coll attribute.

In [10]:
# tbd: create recipe/mapping with different data layers
import semantique as sq

# define an empty data cube
with open("../gsemantique/data/layout.json", "r") as file:
    dc = sq.datacube.STACCube(
        json.load(file), 
        src=[]
    )

# define exemplary mapping
mapping = sq.mapping.Semantique()
mapping["entity"] = {}
valid_indices = [
    dc.layout["Planet"]["reflectance"]["lndst_qa"]["labels"][x]
    for x in ["clear_land_etm", "clear_water_etm", "clear_land_oli", "clear_water_oli"]
]
mapping["entity"]["cloudfree"] = {
    "color": sq.layer("Planet", "reflectance", "lndst_qa").evaluate("in", valid_indices)
}
mapping["entity"]["water"] = {
    "color": (
        sq.layer("Planet", "reflectance", "lndst_green")
            .evaluate("normalized_difference", sq.layer("Planet", "reflectance", "lndst_nir08"))
            .evaluate("greater_equal", 0.2)
    )
}
mapping["entity"]["vegetation"] = {
    "color": (
        sq.layer("Planet", "reflectance", "lndst_nir08")
            .evaluate("normalized_difference", sq.layer("Planet", "reflectance", "lndst_red"))
            .evaluate("greater_equal", 0.2)
    )
}

# define exemplary recipe
recipe = sq.QueryRecipe()
recipe["cloudfree"] = (
    sq.entity("cloudfree")
    .reduce("count", "time")
)
recipe["water_frequency"] = (
    sq.entity("water")
    .filter(sq.entity("cloudfree"))
    .reduce("percentage", "time")
)

fdr.search_auto(recipe, mapping, dc)
fdr.item_coll

The recipe references the following data layers:
('Planet', 'reflectance', 'lndst_green')
('Planet', 'reflectance', 'lndst_qa')
('Planet', 'reflectance', 'lndst_nir08')


Found 1 datasets
Found 1 datasets
Found 1 datasets


Note that items will be retrieved multiple times, if the recipe references different assets of the same item. The results are then consolidated automatically by merging assets on an item level. In the example above, the three different assets all belong to items from the landsat collection.

## Downloading data 

For reason such as data sovereignty, performance improvements for repeated executions or the ability to explore the input data in common GIS systems it may be desirable not to pull the input data for the STACCube from the network for each recipe evaluation, but to persist the data locally. gsemantique offers the `Downloader` class for this purpose, which downloads the results of the `Finder` and generates a corresponding STAC-conformant ItemCollection with the local asset pointers. Specifically, you can instantiate a `Downloader` by forwarding a Finder result to it. Calling the .run() method will start the download process and provide some indication of its progress. You can specify the output directory if desired.

In [11]:
from gsemantique.data.download import Downloader
?Downloader

Init signature: Downloader(item_coll, out_dir=None)
Docstring:      <no docstring>
Init docstring:
Downloader class tailored to the needs of handling single &
multi-collection outputs as generated by the Finder. Results will
be downloaded & a STAC-conformant metadata description (catalog,
collection and item level) with corresponding relative local links
will be established.

Note: The created STAC metadata json are containing all the originally
provided information of the input item_coll, incl. for example the extra
asset field "semantique:key". Apart from the updated links, the STAC
metadata json is therefore equivalent to the input item_coll.

Args:
    item_coll (pystac.ItemCollection or list of pystac.item.Item):
        The Finder result (search.py) to be downloaded.
    out_dir (str): The directory to download the files to. If not specified,
        a new directory will be created with the current timestamp.
File:           c:\users\felix\repositories\gsemantique\gsemantique\dat

In [12]:
item_coll = fdr.item_coll
out_dir = "files/data"

dwn = Downloader(item_coll, out_dir)
dwn.run()

landsat-c2-l2 (collection 1/1)
Not enough items to estimate size. Skipping preview run.


But what happens if you adapt or extend your recipe in a way that additional data layers that have not been downloaded are referenced now? You can simply forward the novel `Finder` result to the `Downloader` and specify the existing data directory. This way, the new data layers / assets will be added without re-downloading the existing ones. The locally saved STAC metadata json will be updated correspondingly.   

In the example below, the existing recipe is extended by an entity "vegetation" which references the RED band additionally to the NIR band that has already been downloaded before.      

In [13]:
# adapted recipe counting vegetation instead of water
recipe = sq.QueryRecipe()
recipe["cloudfree"] = (
    sq.entity("cloudfree")
    .reduce("count", "time")
)
recipe["vegetation_frequency"] = (
    sq.entity("vegetation")
    .filter(sq.entity("cloudfree"))
    .reduce("percentage", "time")
)

# re-run finder
fdr.search_auto(recipe, mapping, dc)
fdr.item_coll

# re-run download
dwn = Downloader(fdr.item_coll, "data")
dwn.run()

The recipe references the following data layers:


('Planet', 'reflectance', 'lndst_red')
('Planet', 'reflectance', 'lndst_qa')
('Planet', 'reflectance', 'lndst_nir08')
Found 1 datasets
Found 1 datasets
Found 1 datasets


landsat-c2-l2 (collection 1/1)
Not enough items to estimate size. Skipping preview run.


Note that the local catalog contains relative links, such that shifting the data directory to other places doesn't break the links. If you want to re-read the item collection from the disk to use it for further processing, you need to make the references absolute again as shown below. 

In [14]:
# re-read requires to convert files to absolute references
import pystac
catalog = pystac.Catalog.from_file(os.path.join(out_dir, "catalog.json"))
catalog.make_all_asset_hrefs_absolute()
item_list = [x for x in catalog.get_items(recursive=True)]
local_coll = pystac.ItemCollection(item_list)
local_coll

In [ ]:
# clean-up
import shutil
shutil.rmtree(out_dir)

## Accessing data

The search result of items can be forwarded to the STACCube constructor as usual. For further descriptions, find the [semantique documentation](https://zgis.github.io/semantique/_notebooks/datacube.html#The-STACCube-configuration).

In [15]:
import geopandas as gpd
import warnings
from semantique.processor.utils import parse_extent

# construct STACCube with relevant items
with open("../gsemantique/data/layout.json", "r") as file:
    dc = sq.datacube.STACCube(
        json.load(file), 
        src = item_coll,
        group_by_solar_day=True,
        dask_params=None,
    )

# define spatio-temporal extent
res = 0.001
epsg = 4326
time = sq.TemporalExtent(pd.Timestamp(fdr.params_search["t_start"]), pd.Timestamp(fdr.params_search["t_end"]))
space = sq.SpatialExtent(gpd.GeoDataFrame(geometry=[fdr.params_search["aoi"]], crs="EPSG:4326"))
extent = parse_extent(space, time, spatial_resolution = [-res, res], crs = epsg)

# load data via retrieve 
with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    data = dc.retrieve(*('Planet', 'reflectance', 'lndst_nir08'), extent=extent)

data

<xarray.DataArray 'nir08' (time: 1, y: 50, x: 50)> Size: 10kB
array([[[16180., 16329., 16466., ..., 17368., 17571., 16954.],
        [16342., 15620., 16201., ..., 17874., 17778., 17195.],
        [16055., 15389., 16938., ..., 18544., 18710., 17875.],
        ...,
        [17164., 17529., 17791., ..., 16706., 17451., 17804.],
        [17344., 17342., 17680., ..., 17966., 17748., 17621.],
        [17537., 17338., 17350., ..., 17822., 18097., 17922.]]],
      dtype=float32)
Coordinates:
  * x              (x) float64 400B 142.1 142.1 142.1 ... 142.2 142.2 142.2
  * y              (y) float64 400B -34.15 -34.15 -34.15 ... -34.2 -34.2 -34.2
    temporal_ref   int32 4B 0
  * time           (time) datetime64[ns] 8B 2020-01-04
    spatial_feats  (y, x) float64 20kB 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
Attributes:
    spec:        RasterSpec(epsg=4326, bounds=(142.13, -34.2, 142.18, -34.15)...
    crs:         epsg:4326
    transform:   | 0.00, 0.00, 142.13|\n| 0.00,-0.00,-34.15|\n| 0.00, 0.00, 1...
    resolution:  0.001
    value_type:  continuous

## Adding new datasets

Gsemantique is set up to be expandable such that data sets beyond the pre-configured content of the existing dataset catalog can be used. The process of how to add new data sets to the catalog is described below: 

First, an instance of the `Dataset` class needs to be created. Data product characteristics can be manually entered when doing so. The minimum set of parameters that needs to be specified includes "provider", "endpoint" and "collection" and "temporality". If not specified, the spatio-temporal extent will be automatically inferred by querying the corresponding STAC properties of the collection. Even though not strictly needed, it is advisable to specify the remaining attributes ("category", "src", "info" and "copyright").

In [16]:
ds = Dataset(
    provider="Planet",
    endpoint="https://planetarycomputer.microsoft.com/api/stac/v1",
    collection="alos-fnf-mosaic",
    temporality="Y",
    temporal_extent=None,
    spatial_extent=None,
    category="SAR",
    src="https://planetarycomputer.microsoft.com/dataset/alos-fnf-mosaic",
    info="Global forest/non-forest maps  (FNF) based on 25m resolution SAR datasets generated by JAXA using the L-band Synthetic Aperture Radar sensors on the Advanced Land Observing Satellite-2 (ALOS-2 PALSAR-2), the Advanced Land Observing Satellite (ALOS PALSAR) and the Japanese Earth Resources Satellite-1 (JERS-1 SAR). FNF maps were generated by a Random Forest machine learning-based classification, Forest is defined as the tree covered land with an area larger than 0.5 ha and a canopy cover of over 10 %, in accordance with the FAO definition of forest.",
    copyright="Jaxa terms of use of research data",
)

As explained in the first section of this notebook, Dataset instances summarise information on the collection level whereas the data layer specific information needs to be added in the layout.json. Referencable assets need to be entered there with their characteristics as described in the [semantique documentation](https://zgis.github.io/semantique/_notebooks/datacube.html#The-STACCube-configuration). If the structure of the data at hand is unknown, an exemplary request can be carried out as shown below to investigate the available assets before entering them in the layout.json.

In [21]:
# make an exemplary search to investigate the structure of the data
import planetary_computer as pc
from pystac_client import Client

catalog = Client.open(ds.endpoint, modifier=pc.sign_inplace)
query = catalog.search(
    collections=ds.collection,
    datetime=ds.temporal_extent,
    max_items=1,
)
item_coll = query.item_collection()
item_coll[0]

<Item id=S56W072_20_FNF>

As you can see from the search result above, there is one relevant data layer named "C". This data layer contains the classification forest vs. non-forest with a total of 5 classes. The corresponding information can also been drawn from the [original data documentation](https://planetarycomputer.microsoft.com/dataset/alos-fnf-mosaic). Along with dtype information, this needs to be entered in a layout.json file as done with this [one](files/layout_ext.json). The layout reference key - here (Planet, classification, forest) - can be choosen freely. Adhering to the organisation of the layout.json for the [existing data catalog](../gsemantique/data/layout.json), the dataset provider (Planetary) is used as top-level key to avoid ambiguities in cases where the same data layer may be offered by different providers.

Next, the connection between the Dataset instance and novel layout parts is established by providing the attributes layout file and layout keys during an .add_layout_info call. The dataset instance can then be added to the overall data catalog.

In [24]:
ds.add_layout_info(
    keys = [("Planet", "classification", "forest")],
    file = os.path.abspath("files/layout_ext.json")
)
ds_catalog.add(ds)

To persist the data set as part of the data catalogue for future use you can call the .save() method and reload it from the pickled file next time. Per default, the data catalog is retrieved from a pickled file stored as part of the package instead. Its advisable to leave the later unaltered and save the new/extended data catalogue in a different location.   

In [29]:
# persist data catalog locally
ds_catalog.save("files/new_catalog.pkl")

# re-load & show extended data catalog
ds_catalog = DatasetCatalog()
ds_catalog.load(cache_path="files/new_catalog.pkl")
ds_catalog.parse_as_table(keys=None)

,category,collection,copyright,endpoint,info,layout_bands,layout_file,layout_keys,provider,spatial_extent,src,temporal_extent,temporality,n_bands
0,SAR,sentinel-1-rtc,CC BY 4.0,https://planetarycomputer.microsoft.com/api/st...,Sentinel-1 represent radar imaging (SAR) satel...,"{'s1_amp_vv': {'name': 'vv', 'description': 'G...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, reflectance, s1_amp_vv), (Planet, re...",Planet,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2014-10-10 00:28:21+00:00, None]",s,4
1,multispectral,sentinel-2-l2a,Copernicus Sentinel Data Terms,https://planetarycomputer.microsoft.com/api/st...,The Sentinel-2 program provides global imagery...,"{'s2_band01': {'name': 'B01', 'description': '...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, reflectance, s2_band01), (Planet, re...",Planet,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2015-06-27 10:25:31+00:00, None]",s,13
2,multispectral,landsat-c2-l2,Public Domain (https://www.usgs.gov/emergency-...,https://planetarycomputer.microsoft.com/api/st...,"Landsat Collection 2 Level-2 Science Products,...","{'lndst_coastal': {'name': 'coastal', 'descrip...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, reflectance, lndst_coastal), (Planet...",Planet,"[-180.0, -90.0, 180.0, 90.0]",https://planetarycomputer.microsoft.com/datase...,"[1982-08-22 00:00:00+00:00, None]",s,10
3,landcover,esa-worldcover,Creative Commons Attribution 4.0 International...,https://planetarycomputer.microsoft.com/api/st...,The European Space Agency (ESA) WorldCover pro...,"{'esa_lc': {'name': 'map', 'description': 'ESA...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, classification, esa_lc)]",Planet,"[-180.0, -60.0, 180.0, 83.0]",https://planetarycomputer.microsoft.com/datase...,"[2020-01-01 00:00:00+00:00, 2021-12-31 23:59:5...",Y,1
4,landcover,io-lulc-annual-v02,Creative Commons BY-4.0,https://planetarycomputer.microsoft.com/api/st...,Time series of annual global maps of land use ...,"{'impact_lc': {'name': 'data', 'description': ...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, classification, impact_lc)]",Planet,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2017-01-01 00:00:00+00:00, 2024-01-01 00:00:0...",Y,1
5,DEM,nasadem,Public Domain (https://lpdaac.usgs.gov/data/da...,https://planetarycomputer.microsoft.com/api/st...,NASADEM provides global topographic data at 1 ...,"{'dem': {'name': 'elevation', 'description': '...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, topography, dem)]",Planet,"[-179.0, -56.0, 179.0, 61.0]",https://planetarycomputer.microsoft.com/datase...,"[2000-02-20 00:00:00+00:00, 2000-02-20 00:00:0...",None,1
6,DSM,cop-dem-glo-30,None,https://planetarycomputer.microsoft.com/api/st...,The Copernicus DEM is a digital surface model ...,"{'dsm': {'name': 'data', 'description': 'Digit...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, topography, dsm)]",Planet,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2021-04-22 00:00:00+00:00, 2021-04-22 00:00:0...",None,1
7,fire detection,modis-64A1-061,None,https://planetarycomputer.microsoft.com/api/st...,The Terra and Aqua combined MCD64A1 Version 6....,"{'m_burn_date': {'name': 'Burn_Date', 'descrip...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, burned_mapping, m_burn_date), (Plane...",Planet,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2000-11-01 00:00:00+00:00, None]",M,3
8,fire detection,modis-14A2-061,None,https://planetarycomputer.microsoft.com/api/st...,The Moderate Resolution Imaging Spectroradiome...,"{'w_burn_qa': {'name': 'QA', 'description': 'P...",c:\users\felix\repositories\gsemantique\gseman...,"[(Planet, burned_mapping, w_burn_qa), (Planet,...",Planet,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2000-02-18 00:00:00+00:00, Non

Finally, note that `STACCube` requires assets to be organised as single-band products, otherwise the stacking process will fail. Search results from non-conformant datasets with an atypical organisation of items/assets may need to be transformed before forwarding them to the `STACCube` constructor. An example for this is the SAR data set "sentinel-1-global-coherence" as contained in the pre-configured data catalog. Since data layers are organised as items instead of assets here, a custom postprocessing procedure is implemented for this data set in the [Finder](../gsemantique/data/search.py). For our forest dataset example, this is not needed as the data layer is already organised as an asset (adhering to the common standards of STAC data organisation). 